In [302]:
! pip install opencv-python opencv-python-headless

import numpy as np
from PIL import Image
import scipy
import cv2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 MB 63.6 MB/s eta 0:00:0000:0100:01


In [43]:
def normalize(p):
    return p / np.sum(p)

In [44]:
def dist(x, y, alpha):
    return ((x[0]-y[0])**alpha + (x[1]-y[1])**alpha)

In [45]:
def Cost_matrix(N, alpha):
    C = np.zeros((N*N, N*N))
    for j1 in range(N):
        for j2 in range(N):
            column = []
            for i in range(N):
                column.append(dist([0, j1], [i, j2], alpha))
            C[N*j1:N*j1+N, N*j2:N*j2+N] = scipy.linalg.toeplitz(np.array(column))
    return(C)

In [405]:
def multiply(K, X): # Takes in argument a matrix of cost K and a vector X
    n = int(np.sqrt(len(K[0])))
    Y = np.zeros(len(K[0]))
    for j in range(n):
        for i in range(n):
            Y[n*j:n*j+n] += scipy.linalg.matmul_toeplitz(K[n*i:n*i+n, n*j:n*j+n][0], X[n*j:n*j+n])
    return Y

In [479]:
def divide(P, Q):
    Q_new = Q.copy()
    P_new = P.copy()
    for i in np.argwhere(Q==0):
        if P[i]==0:
            Q_new[i]=1
            P_new[i]=1
    return(P/Q_new)

In [480]:
def inner_log(X, Y):
    X_new = X.copy()
    for i in np.argwhere(np.abs(X)==0):
        if Y[i]==0:
            X_new[i]=1          
    return np.inner(np.log(X_new), Y)

In [481]:
def sinkhorn_differentiate(Lambda, C, p, q, Gamma, L):

    S = len(p)
    N = len(p[0])
    K = np.exp(-C/Gamma)
    Phi = np.ones((L+1, N, S))
    b = np.ones((L+1, N, S))
    w = np.zeros(S)
    r = np.zeros((N, S))
    Lambda0 = normalize(np.exp(np.copy(Lambda)))
    
    for i in range(1, L+1):
        P = np.ones(N)

        for j in range(S):
            if i==2:
                print(b[i-1, :, j], p[j])
            Phi[i, :, j] = multiply(K, divide(p[j], multiply(K, b[i-1, :, j])))
            P *= (np.abs(Phi[i, :, j]))**Lambda0[j]
            
        for j in range(S):
            b[i, :, j] = divide(P, Phi[i, :, j])

    g = 2*(P - q) * P
    
    for i in range(1, L+1):

        for j in range(S):
            w[j] += inner_log(Phi[L-i+1, :, j], g)
            r[:, j] = - multiply(K, (multiply(K, divide((Lambda0[j] * g - r[:, j]), Phi[L-i+1, :, j])) * divide(p[j], multiply(K, b[L-i, :, j]) ** 2))) * b[L-i, :, j]
       
        g = np.zeros(N)
        for j in range(S):    
            g += r[:, j]

    return P, w

In [482]:
def histogram_colors(img_path):
    img = cv2.imread(img_path)
    ycrcb_img = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    Y, Cr, Cb = cv2.split(ycrcb_img)
    hist_Y = cv2.calcHist([Y], [0], None, [128], [0, 256])
    hist_CrCb = cv2.calcHist([Cr, Cb], [0, 1], None, [128, 128], [0, 256, 0, 256])
    return hist_CrCb.reshape(-1)

In [483]:
hist1 = histogram_colors('Images/ML_HOGGAR/HOGGAR_data1.jpg')
hist2 = histogram_colors('Images/ML_HOGGAR/HOGGAR_data2.jpg')
hist3 = histogram_colors('Images/ML_HOGGAR/HOGGAR_data3.jpg')
hists = np.array([hist1, hist2, hist3])

q = histogram_colors('Images/ML_HOGGAR/HOGGAR_obj.jpg')

In [484]:
N = len(hists[0])
C = Cost_matrix(int(np.sqrt(N)), 2)
Lambdas = np.array([1 for i in range(len(hists))])
sinkhorn_differentiate(Lambdas, C, hists, q, 1, 10)

[0. 0. 0. ... 0. 0. 0.] [0. 0. 0. ... 0. 0. 0.]


/tmp/ipykernel_888/3589161652.py:8: RuntimeWarning: divide by zero encountered in divide
  return(P/Q_new)


[0. 0. 0. ... 0. 0. 0.] [0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.] [0. 0. 0. ... 0. 0. 0.]


/tmp/ipykernel_888/2826033818.py:6: RuntimeWarning: invalid value encountered in log
  return np.inner(np.log(X_new), Y)
/tmp/ipykernel_888/2826033818.py:6: RuntimeWarning: divide by zero encountered in log
  return np.inner(np.log(X_new), Y)


(array([0., 0., 0., ..., 0., 0., 0.], shape=(16384,)), array([nan, nan, nan]))

In [427]:
Lambda0 = normalize(np.exp(np.copy(Lambdas)))
print(Lambda0)

[0.33333333 0.33333333 0.33333333]


In [266]:
p = hists
LambdaINI = np.array([0.3, 0.4, 0.3])
resH = scipy.optimize.minimize(sinkhorn_differentiate, LambdaINI, method='BFGS', args=(p, q, 10, 10), jac=True)

In [267]:
print(resH)

  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 0.002414900630069163
        x: [ 1.383e+00 -1.119e+00 -6.910e-01]
      nit: 19
      jac: [ 1.032e-04 -3.148e-04  1.027e-04]
 hess_inv: [[ 1.399e+03  2.494e+01  5.439e+03]
            [ 2.494e+01  1.346e+02  5.489e+02]
            [ 5.439e+03  5.489e+02  2.269e+04]]
     nfev: 80
     njev: 69


In [268]:
LambdaH = normalize(np.exp(resH.x))
print(LambdaH)

[0.82804178 0.06784739 0.10411083]


In [295]:
def sinkhorn(Lambda, hists, input_image, Gamma, L, epsilon):

    N = len(hists[0])
    C = Cost_matrix(int(np.sqrt(N)), 2)
    q, w = sinkhorn_differentiate(Lambda, hists, input_image, Gamma, L)
    K = np.exp(-C/Gamma)
    
    u = np.ones(len(C[0]))
    v = np.ones(len(C[1]))
    T = np.diag(u.flatten()) @ K @ np.diag(v.flatten())
    t_norm = np.trace(T.T @ T)
    
    while True:
        u = input_image/multiply(K, v) # avoid divided by zero
        v = q/multiply(K, u)
        T = np.diag(u.flatten()) @ K @ np.diag(v.flatten())
        if abs((np.trace(T.T @ T) - t_norm)/t_norm) < epsilon:
            break
        t_norm = np.trace(T.T @ T)
    return T, np.trace(C.T @ T)

In [299]:
sinkhorn(LambdaH, hists, input_image, 10, 10, 1)

(array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [6.36341830e-04, 5.78886013e-04, 4.95646244e-04, ...,
         1.08443812e-20, 7.04137841e-22, 4.47467756e-23],
        [6.71645542e-05, 7.46279680e-05, 7.80439562e-05, ...,
         1.54106194e-20, 1.22217078e-21, 9.48625633e-23],
        ...,
        [1.69143699e-21, 1.69615355e-20, 1.60085016e-19, ...,
         3.57280777e-05, 2.55722719e-05, 1.79134870e-05],
        [3.92077459e-22, 4.80219854e-21, 5.53585272e-20, ...,
         1.11504232e-04, 9.74787413e-05, 8.34026073e-05],
        [1.58525182e-22, 2.37151237e-21, 3.33909447e-20, ...,
         6.06992594e-04, 6.48128097e-04, 6.77313076e-04]], shape=(256, 256)),
 np.float64(7.397769835943807))

In [300]:
def transformation(x, T):
    

SyntaxError: incomplete input (4061780513.py, line 2)

In [496]:
L = 5
Gamma = 1
p = hists
S = len(p)
N = len(p[0])
K = np.exp(-C/Gamma)
Phi = np.ones((L+1, N, S))
b = np.ones((L+1, N, S))
w = np.zeros(S)
r = np.zeros((N, S))
Lambda0 = normalize(np.exp(np.copy(Lambdas)))
    
for i in range(1, L+1):
    P = np.ones(N)

    for j in range(S):
        #if i==1 and j==0:
            #A = multiply(K, b[i-1, :, j])
        Phi[i, :, j] = multiply(K, divide(p[j], multiply(K, b[i-1, :, j])))
        P *= (np.abs(Phi[i, :, j]))**Lambda0[j]
        if i==1 and j==0: 
            A=P
        
    for j in range(S):
        b[i, :, j] = divide(P, Phi[i, :, j])

/tmp/ipykernel_888/3589161652.py:8: RuntimeWarning: divide by zero encountered in divide
  return(P/Q_new)


In [497]:
np.count_nonzero(A==0)

11904

In [491]:
np.count_nonzero(p[0]==0)

14610

In [493]:
Phi[2, :, 0] = multiply(K, divide(p[0], A))

/tmp/ipykernel_888/3589161652.py:8: RuntimeWarning: divide by zero encountered in divide
  return(P/Q_new)


In [499]:
print(A.reshape(128, 128))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
